In [3]:
import tensorflow as tf
import google.protobuf

print("TensorFlow version:", tf.__version__)
print("protobuf version:", google.protobuf.__version__)


TensorFlow version: 2.18.0
protobuf version: 5.29.3


In [ ]:
from tensorflow.keras.models import load_model

# 기존 모델 로드
model = load_model(r'C:\baekjoon_Git_repository\face-emotion-recognition\models\affectnet_emotions\mobilenet_finetuned3.h5')
# 모델 구조 확인
model.summary()


Model: "model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_1 (InputLayer)            │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_pad (ZeroPadding2D)       │ (None, 225, 225, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 112, 112, 32)   │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 112, 112, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 112, 112, 32)   │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 112, 112, 64)   │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 56, 56, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 56, 56, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 56, 56, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 3,493,065 (13.32 MB)

 Trainable params: 264,199 (1.01 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

 Optimizer params: 2 (12.00 B)

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.3,  # 0.2 → 0.3
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7,1.3],
    fill_mode="nearest"
)


val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    "D:/train/Training",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    "D:/validation",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

steps_per_epoch = 40000 // 32
validation_steps = 4000 // 32  # 검증 데이터에서 사용할 샘플 수


with strategy.scope():
    model = load_model("C:/baekjoon_Git_repository/face-emotion-recognition/models/affectnet_emotions/mobilenet_7.h5", compile=False)

    # 마지막 몇 개의 층만 학습 가능하도록 설정 (Fine-Tuning)
    for layer in model.layers[:-5]:  # 마지막 5개 층만 학습
        layer.trainable = False

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, clipvalue=1.0),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

model.summary()

# 
EPOCHS = 10

# 모델 학습 
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

# 모델 저장
model.save("C:/baekjoon_Git_repository/face-emotion-recognition/models/affectnet_emotions/mobilenet_finetuned3.h5",include_optimizer=True)
print("Fine-Tuning 완료")

⚠️ TPU를 찾을 수 없습니다. GPU/CPU로 실행합니다.
💡 실행 전략: <tensorflow.python.distribute.distribute_lib._DefaultDistributionStrategy object at 0x000001FECCCB3910>
Found 71326 images belonging to 7 classes.
Found 44625 images belonging to 7 classes.


Model: "model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_1 (InputLayer)            │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_pad (ZeroPadding2D)       │ (None, 225, 225, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 112, 112, 32)   │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 112, 112, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 112, 112, 32)   │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 112, 112, 64)   │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 56, 56, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 56, 56, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 56, 56, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 3,493,063 (13.32 MB)

 Trainable params: 1,326,087 (5.06 MB)

 Non-trainable params: 2,166,976 (8.27 MB)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4746s 8s/step - accuracy: 0.1699 - disgust_precision: 0.0000e+00 - fear_precision: 0.0000e+00 - loss: 2.0439 - neutral_precision: 0.0000e+00 - precision: 0.0000e+00 - sadness_precision: 0.0000e+00 - val_accuracy: 0.1462 - val_disgust_precision: 0.0000e+00 - val_fear_precision: 0.0000e+00 - val_loss: 1.6413 - val_neutral_precision: 0.0000e+00 - val_precision: 0.0000e+00 - val_sadness_precision: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 2/10
490/625 ━━━━━━━━━━━━━━━━━━━━ 10:29 5s/step - accuracy: 0.1683 - disgust_precision: 0.0000e+00 - fear_precision: 0.0000e+00 - loss: 1.9238 - neutral_precision: 0.0000e+00 - precision: 0.0000e+00 - sadness_precision: 0.0000e+00

c:\Users\rkgus\.conda\envs\myenv_cpu\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


625/625 ━━━━━━━━━━━━━━━━━━━━ 2616s 4s/step - accuracy: 0.1684 - disgust_precision: 0.0000e+00 - fear_precision: 0.0000e+00 - loss: 1.9239 - neutral_precision: 0.0000e+00 - precision: 0.0000e+00 - sadness_precision: 0.0000e+00 - val_accuracy: 0.1527 - val_disgust_precision: 0.0000e+00 - val_fear_precision: 0.0000e+00 - val_loss: 1.7875 - val_neutral_precision: 0.0000e+00 - val_precision: 0.0000e+00 - val_sadness_precision: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3272s 5s/step - accuracy: 0.1669 - disgust_precision: 0.0000e+00 - fear_precision: 0.0000e+00 - loss: 1.9228 - neutral_precision: 0.0000e+00 - precision: 0.0000e+00 - sadness_precision: 0.0000e+00 - val_accuracy: 0.1245 - val_disgust_precision: 0.0000e+00 - val_fear_precision: 0.0000e+00 - val_loss: 1.8521 - val_neutral_precision: 0.0000e+00 - val_precision: 0.0000e+00 - val_sadness_precision: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2621s 4s/step - acc

✅ Fine-Tuning 완료 및 모델 저장됨!


/////////

/////////

In [8]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ✅ 기존 학습된 모델 불러오기
model = load_model('fine_tuned_model.h5')

# ✅ 일부 층만 학습 가능하도록 설정 (Fine-Tuning)
for layer in model.layers[:-5]:  # 마지막 5개 층만 학습
    layer.trainable = False

# ✅ 모델 컴파일 (이전과 동일한 설정 유지)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, clipvalue=1.0),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# ✅ 새롭게 추가 학습할 데이터 로드
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32  

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# ✅ 새로운 학습 데이터 경로 (추가 데이터 사용)
new_train_dir = "D:/train/Training"
new_val_dir = "D:/validation"

train_generator = train_datagen.flow_from_directory(
    new_train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    new_val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# ✅ 사용할 데이터 양 설정
steps_per_epoch = 10000 // BATCH_SIZE  # 학습 샘플 개수 제한
validation_steps = 1000 // BATCH_SIZE  # 검증 샘플 개수 제한

# ✅ 추가 학습 진행
EPOCHS = 5  # 추가 학습이므로 epoch 수를 너무 높이지 않음
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

# ✅ 추가 학습된 모델 저장
model.save(".h5")
print("✅ 추가 학습 완료 및 모델 저장됨!")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'fine_tuned_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
#모델저장
fine_tuned_model.save('fine_tuned_model.h5')

In [ ]:
import tensorflow as tf
import scipy.sparse.linalg
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ✅ 기존 학습된 모델 불러오기
model = load_model('C:/baekjoon_Git_repository/face-emotion-recognition/models/affectnet_emotions/mobilenet_finetuned_1.h5')

# ✅ 일부 층만 학습 가능하도록 설정 (Fine-Tuning)
for layer in model.layers[:-5]:  # 마지막 5개 층만 학습
    layer.trainable = False

# ✅ 모델 컴파일 (이전과 동일한 설정 유지)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, clipvalue=1.0),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# ✅ 새롭게 추가 학습할 데이터 로드
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32  

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# ✅ 새로운 학습 데이터 경로 (추가 데이터 사용)
new_train_dir = "D:/train/Training"
new_val_dir = "D:/validation"

train_generator = train_datagen.flow_from_directory(
    new_train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    new_val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# ✅ 사용할 데이터 양 설정
steps_per_epoch = 30000 // BATCH_SIZE  # 학습 샘플 개수 제한
validation_steps = 3000 // BATCH_SIZE  # 검증 샘플 개수 제한

# ✅ 추가 학습 진행
EPOCHS = 5  # 추가 학습이므로 epoch 수를 너무 높이지 않음
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

# ✅ 추가 학습된 모델 저장
model.save("C:/baekjoon_Git_repository/face-emotion-recognition/models/affectnet_emotions/movilenet_finetuned_2.h5")
print("✅ 추가 학습 완료 및 모델 저장됨!")


Found 83138 images belonging to 7 classes.
Found 44625 images belonging to 7 classes.
Epoch 1/5
937/937 ━━━━━━━━━━━━━━━━━━━━ 2627s 3s/step - accuracy: 0.1450 - loss: 1.9747 - val_accuracy: 0.1717 - val_loss: 1.9467
Epoch 2/5
937/937 ━━━━━━━━━━━━━━━━━━━━ 2368s 3s/step - accuracy: 0.1475 - loss: 1.9658 - val_accuracy: 0.1573 - val_loss: 1.9497
Epoch 3/5
725/937 ━━━━━━━━━━━━━━━━━━━━ 7:46 2s/step - accuracy: 0.1515 - loss: 1.9611

c:\Users\rkgus\.conda\envs\myenv_cpu\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


937/937 ━━━━━━━━━━━━━━━━━━━━ 1841s 2s/step - accuracy: 0.1503 - loss: 1.9611 - val_accuracy: 0.1613 - val_loss: 1.9496
Epoch 4/5
937/937 ━━━━━━━━━━━━━━━━━━━━ 2319s 2s/step - accuracy: 0.1465 - loss: 1.9584 - val_accuracy: 0.1767 - val_loss: 1.9473
Epoch 5/5
937/937 ━━━━━━━━━━━━━━━━━━━━ 2312s 2s/step - accuracy: 0.1425 - loss: 1.9553 - val_accuracy: 0.1687 - val_loss: 1.9488


✅ 추가 학습 완료 및 모델 저장됨!
